In [ ]:
pip install -r ./requirements.txt

In [2]:
import os
import pandas as pd
import shutil

In [ ]:
%cd ../py

from constants import (
    CSV_EXTRACTED_PATH,
    CSV_MERGED_PATH,
    DATA_LARGE_KEYS,
    DATA_PATH,
    TYPES,
    YEARS,
)
from file.file_data_write import file_data_write

%cd ../notebook

In [4]:
shutil.rmtree(CSV_EXTRACTED_PATH)
shutil.rmtree(CSV_MERGED_PATH)
shutil.rmtree(DATA_PATH)

In [ ]:
!bash ../scripts/unzip.sh

In [6]:
os.mkdir(CSV_MERGED_PATH)

In [7]:
for type in TYPES.keys():
    df = pd.DataFrame()

    for file in os.listdir(CSV_EXTRACTED_PATH):
        if file.endswith(f"{type}.csv"):
            file_df = pd.read_csv(
                f"{CSV_EXTRACTED_PATH}/{file}",
                encoding="latin1",
                engine="python",
            )

            file_df = file_df[TYPES[type].values()]

            if df.empty:
                df = file_df
            else:
                df = pd.concat([df, file_df])

    df.to_csv(
        f"{CSV_MERGED_PATH}/{type}.csv",
        encoding="latin1",
        index=False,
        sep=",",
    )

In [8]:
os.mkdir(DATA_PATH)

In [ ]:
%cd ../py

from df import df
from filter.filter import filter
from main import main

%cd ../notebook

In [10]:
dfs = df()

In [15]:
exclude_ids = []

for key in DATA_LARGE_KEYS.keys():
    data_key = DATA_LARGE_KEYS[key]

    file_data_write(main(*data_key["filter"]), key)

    filtered_dfs = filter(*data_key["filter"])

    if data_key["exclude"]:
        exclude_ids = (
            exclude_ids + filtered_dfs["df_DATA"][TYPES["DATA"]["id"]].tolist()
        )

for type in TYPES.keys():
    df_type = dfs[f"df_{type}"]
    df_type = df_type[~df_type[TYPES[type]["id"]].isin(exclude_ids)]
    df_type.to_csv(
        f"{CSV_MERGED_PATH}/{type}.csv",
        encoding="latin1",
        index=False,
        sep=",",
    )

0
527209
995221
2051121
